# HCFullPipeline Walkthrough

> Deep-dive into the master pipeline: stages, checkpoints, agents, and recovery.

---

In [ ]:
import requests, json, os, yaml

HEADY_ENDPOINT = os.getenv('HEADY_ENDPOINT', 'http://localhost:3300')
HEADY_API_KEY = os.getenv('HEADY_API_KEY', '')
headers = {'x-api-key': HEADY_API_KEY} if HEADY_API_KEY else {}

def api(path, method='GET', data=None):
    url = f'{HEADY_ENDPOINT}{path}'
    if method == 'GET':
        return requests.get(url, headers=headers, timeout=15).json()
    return requests.post(url, headers=headers, json=data, timeout=30).json()

print(f'Connected to: {HEADY_ENDPOINT}')

## 1. Pipeline Configuration

The pipeline is defined in `configs/hcfullpipeline.yaml`. Let's inspect it.

In [ ]:
config = api('/api/pipeline/config')
print(json.dumps(config, indent=2))

## 2. Stage Dependency Graph (DAG)

```
ingest → plan → execute-major-phase → recover → finalize
```

Each stage has:
- **checkpoint: true** — system re-analyzes state at this point
- **parallel** — whether tasks within the stage run concurrently
- **dependsOn** — which stages must complete first

In [ ]:
dag = api('/api/pipeline/dag')
print(json.dumps(dag, indent=2))

## 3. Checkpoint Protocol

At each checkpoint, the system:
1. **Validates run state** — pipeline def + resource policies still current
2. **Compares config hashes** — detect drift from repo state
3. **Re-evaluates health** — bottlenecks, errors, spend vs budget
4. **Checks concept alignment** — which patterns active, suggest missing
5. **Applies approved patterns** — gradual enablement at boundaries
6. **Syncs all files** — docs, registry, notebooks updated per Checkpoint Protocol
7. **Reports** — comprehensive status with concept usage + config hashes

In [ ]:
# Run a checkpoint analysis
# checkpoint = api('/api/checkpoint/analyze', 'POST', {'stage': 'manual'})
# print(json.dumps(checkpoint, indent=2))
print('Checkpoint analysis commented out for safety. Uncomment to run.')

## 4. Agent Status

The Supervisor pattern fans work out to registered agents.

In [ ]:
agents = api('/api/supervisor/status')
print(json.dumps(agents, indent=2))

## 5. Pipeline History

Review past pipeline runs, their status, and readiness scores.

In [ ]:
history = api('/api/pipeline/history')
print(json.dumps(history, indent=2))

## 6. Circuit Breakers

Monitor circuit breaker states for external integrations.

In [ ]:
cb = api('/api/pipeline/circuit-breakers')
print(json.dumps(cb, indent=2))

## 7. Stop Rules

The pipeline enforces stop conditions:
- **error_rate > 15%** → enter recovery
- **readiness < 60** → enter recovery
- **critical alarm** → pause and escalate
- **data integrity failure** → halt immediately

> Build aggressively when healthy; repair first when not.

---

## Next Steps

- **Registry deep-dive:** `notebooks/examples/registry-api-demo.ipynb`
- **Quick start:** `notebooks/quick-start/heady-quick-start.ipynb`
- **Full pipeline config:** `configs/hcfullpipeline.yaml`

*This notebook is maintained under the Checkpoint Protocol.*